<a href="https://colab.research.google.com/github/deayalar/deeplearning_unitn/blob/main/DL_Project_wandb_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://market1501.s3-us-west-2.amazonaws.com/dataset.zip
!unzip -q dataset.zip -d dataset

--2021-06-22 14:32:28--  https://market1501.s3-us-west-2.amazonaws.com/dataset.zip
Resolving market1501.s3-us-west-2.amazonaws.com (market1501.s3-us-west-2.amazonaws.com)... 52.218.128.133
Connecting to market1501.s3-us-west-2.amazonaws.com (market1501.s3-us-west-2.amazonaws.com)|52.218.128.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82925180 (79M) [application/zip]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>]  79.08M  18.4MB/s    in 5.6s    

2021-06-22 14:32:34 (14.0 MB/s) - ‘dataset.zip’ saved [82925180/82925180]



In [2]:
!rm -rf /content/deeplearning_unitn
!git clone https://github.com/deayalar/deeplearning_unitn.git

Cloning into 'deeplearning_unitn'...
remote: Enumerating objects: 170, done.
remote: Counting objects: 100% (170/170), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 170 (delta 95), reused 116 (delta 52), pack-reused 0
Receiving objects: 100% (170/170), 5.42 MiB | 11.42 MiB/s, done.
Resolving deltas: 100% (95/95), done.


In [3]:
!nvidia-smi

Tue Jun 22 14:32:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [33]:
config = dict(
    wandb = False,
    device = "auto", # Select an specific device None to select automatically
    train_root = "/content/dataset/train",
    test_root = "/content/dataset/test", 
    queries_root = "/content/dataset/queries",
    attributes_file = "/content/dataset/annotations_train.csv",
    #train_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/train",
    #test_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/test",
    #queries_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/queries",
    #attributes_file = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/annotations_train.csv",
    dataset="Market1501",
    backbone = "resnet18",
    split = dict(
        full_training_size = 0.75,
        train_size = 0.8
    ),
    compose = dict(
        resize_h = 224,
        resize_w = 224
    ),
    epochs=1,
    training_batch_size=8,
    validation_batch_size=32,
    learning_rate=0.01,
    weight_decay=0.000001, 
    momentum=0.9,
    test_before_training=True,
    test_after_epochs=10,
    mAP_rank=10)

In [34]:
%cd /content/deeplearning_unitn

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
from tqdm.notebook import tqdm

import cost_functions
from evaluation import Evaluator
from my_datasets.reid_dataset import Market1501
from cost_functions import AttributesLossWrapper
from utils.split_data import ValidationSplitter, TrainingSplitter
from models.reid_model import FinetunedModel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

if config["device"] == "auto":
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
else:
    device = config["device"]
print(device)

/content/deeplearning_unitn
cuda:0


In [ ]:
!pip install wandb -q
import wandb
wandb.login()

/bin/bash: pip: command not found


ModuleNotFoundError: No module named 'wandb'

In [35]:
def model_pipeline(hyperparameters):
    """
    This function corresponds to the basic pipeline of all tested models
    0) Split data
    1) Setup based on the configuration
    2) Train the model
    3) Test performance
    """
    config = hyperparameters
    if config["wandb"]:
      wandb.init(entity="dl_unitn", project="dl_project", config=hyperparameters)
      config = wandb.config
    print(config)
    
    train_set, val_set, val_queries = split_data(config)
    
    model, train_loader, val_loader, val_queries_loader, criterion, optimizer = setup(train_set, val_set, val_queries, config)
    id_ground_truth_dict = build_ground_truth(val_set, val_queries)

    print("Using "+ config["backbone"] + " as backbone")
    if config["test_before_training"]:
      test(model, val_loader, val_queries_loader, id_ground_truth_dict, config)

    train(model, train_loader, val_loader, criterion, optimizer, config)

    test(model, val_loader, val_queries_loader, id_ground_truth_dict, config, save_model=True)

    return model

In [36]:
def build_ground_truth(val_set, val_queries):
    values = []
    for q in val_queries:
        matches = []
        for idx_v, v in enumerate(val_set):
            if v.split("_")[0] == q.split("_")[0]:
                matches.append(idx_v)
        value = set(matches)
        values.append(value)
        
    ground_truth_dict = dict(zip(list(range(0, len(val_queries))), values))
    return ground_truth_dict


In [37]:
def split_data(config):
    """Returns a list with the names of theimages in each set"""
    splitter = ValidationSplitter(train_root=config["train_root"], 
                                  test_root=config["test_root"], 
                                  queries_root=config["queries_root"])
    train_set, val_set, val_queries = splitter.split(train_size=config["split"]["full_training_size"],
                                                     random_seed=42)
    return train_set, val_set, val_queries

def setup(train_set, val_set, val_queries, config):
    #Create pytorch Datasets 
    composed = transforms.Compose([transforms.Resize((config["compose"]["resize_h"], 
                                                      config["compose"]["resize_w"])),
                                   transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                        std=[0.229, 0.224, 0.225])])
    
    train_dataset = Market1501(root_dir=config["train_root"],
                            attributes_file=config["attributes_file"],
                            #full_train_set=full_train_set,
                            images_list=train_set,
                            transform=composed)
                            
    val_dataset = Market1501(root_dir=config["train_root"],
                         attributes_file=config["attributes_file"],
                         #full_train_set = full_train_set,
                         images_list=val_set,
                         transform=composed)

    val_queries_dataset = Market1501(root_dir=config["train_root"],
                         attributes_file=config["attributes_file"],
                         #full_train_set = full_train_set,
                         images_list=val_queries,
                         transform=composed)

    train_loader = torch.utils.data.DataLoader(train_dataset, 
                                               batch_size=config["training_batch_size"], 
                                               shuffle=True, 
                                               num_workers=8)
                                               
    val_loader = torch.utils.data.DataLoader(val_dataset, 
                                             batch_size=config["validation_batch_size"], 
                                             shuffle=False, 
                                             num_workers=8)

    val_queries_loader = torch.utils.data.DataLoader(val_queries_dataset, 
                                             batch_size=config["validation_batch_size"],
                                             shuffle=False, 
                                             num_workers=8)

    attr_len = len(train_dataset[0][2]) #Number of attributes in the csv: 27
    print(f"Number of attributes: {attr_len}")
    model = FinetunedModel(architecture=config["backbone"] ,n_classes=attr_len).to(device)
    
    criterion = AttributesLossWrapper(2) #TODO: So far it optimizes only the attributes but this loss should be the combination of identification and attributes
    optimizer = torch.optim.SGD(model.parameters(), 
                                lr=config["learning_rate"], 
                                weight_decay=config["weight_decay"], 
                                momentum=config["momentum"])
    
    return model, train_loader, val_loader, val_queries_loader, criterion, optimizer

In [38]:
def train(model, train_loader, val_loader, criterion, optimizer, config):
    # tell wandb to watch what the model gets up to: gradients, weights, and more!
    if config["wandb"]:
        wandb.watch(model, criterion, log="all", log_freq=10)
    model.train()
    # Run training and track with wandb
    total_batches = len(train_loader) * config["epochs"]
    example_ct = 0  # number of seen examples
    batch_ct = 0

    for epoch in tqdm(range(config["epochs"])):
        for batch_idx, (inputs, identity, attributes) in enumerate(train_loader):
            loss, predicted = train_batch(inputs, identity, attributes, model, optimizer, criterion)

            example_ct +=  len(inputs)
            batch_ct += 1

            if ((batch_ct + 1) % 50) == 0:
                train_log(loss, example_ct, epoch)


def train_batch(inputs, identity, attributes, model, optimizer, criterion):
    inputs = inputs.to(device)
    attributes = attributes.to(device)
    
    # Forward pass
    outputs = model(inputs)
    # Apply the loss
    loss = criterion(outputs, attributes)
    
    optimizer.zero_grad()
    # Backward pass
    loss.backward()

    # Step with optimizer
    optimizer.step()

    # Zeros the gradient
    predicted = torch.round(outputs[0].squeeze(1))

    return loss, predicted

In [39]:
def train_log(loss, example_ct, epoch):
    loss = float(loss)
    if config["wandb"]:
        wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    print(f"Epoch {epoch}: Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")

In [40]:
def get_features_from_loader(model, loader):
    model.eval()
    with torch.no_grad():
        all_features = torch.zeros(len(loader.dataset), model.feature_size)
        for batch_idx, (inputs, ids, attr) in enumerate(tqdm(loader)):
                inputs = inputs.to(device)
                features = model(inputs, get_features=True)
                for in_batch, f in enumerate(features):
                    all_features[(batch_idx * loader.batch_size) + in_batch] = f
        return all_features


def test_mAP(model, gallery_loader, queries_loader, ground_truth_dict, config, save_model=False):
    """
    This function returns the mAP performance of a given model 
    Params:
    model: model to be evaluated
    gallery: tensor that contains the feature representations of the target images in validation or test set
    queries: tensor that contains feature representations of the queries
    rank: top number of elements to retrieve

    Returns:
    mAP performance of the model
    """

    # Run the model on some test examples
    with torch.no_grad():
        
        # Compute the features for queries and gallery
        print("Computing gallery features...")
        gallery_features = get_features_from_loader(model, gallery_loader)
        print("Computing query features...")
        query_features = get_features_from_loader(model, queries_loader)
        
        # Build the cosine similarity matrix between the all the queries and all the elements in gallery
        print("Computing cosine similarities...")
        sims_matrix = torch.empty(query_features.size()[0], gallery_features.size()[0])
        for idx, q in enumerate(query_features):
            sims_matrix[idx] = F.cosine_similarity(q, gallery_features, dim=-1)
        
        print("Similarity matrix shape: " + str(sims_matrix.size()))
        sorted_index = torch.argsort(sims_matrix, dim=1, descending=True)
        top_k = sorted_index.narrow_copy(dim=1, start=0, length=config["mAP_rank"])

        #Build the dictionary to compute the mAP
        predictions_dict = {idx:  r for idx, r in enumerate(top_k.tolist())}
        mAP = Evaluator.evaluate_map(predictions_dict, ground_truth_dict)
        
        print(f"mAP: {mAP}")
        if config["wandb"]:
            wandb.log({"mAP": mAP})

In [47]:
def get_attributes_from_loader(model, loader):
    model.eval()

    all_predictions = np.empty(shape=[0, 27], dtype=np.byte)
    all_attrs = np.empty(shape=[0, 27], dtype=np.byte)

    with torch.no_grad():
        for batch_idx, (inputs, ids, attr) in enumerate(tqdm(loader)):
                inputs = inputs.to(device)
                outputs = model(inputs, get_features=False)
                #print("attr:",attr)
                predictions = torch.empty(attr.size()[1], attr.size()[0])
                for attr_idx, output in enumerate(outputs):
                    if output.size()[1] == 1: #If the output is binary
                        pred = torch.round(torch.squeeze(output, 1))
                    else: #Otherwise it is multiclass
                        pred = torch.argmax(output, dim=1)
                    predictions[attr_idx] = pred

                predictions = torch.transpose(predictions, 0, 1).cpu().numpy()
                attr = attr.cpu().numpy()

                all_predictions = np.append(all_predictions, predictions, axis=0)
                #print("all_predictions shape: ", all_predictions.shape)
                all_attrs = np.append(all_attrs, attr, axis=0)
                #print("all_attrs shape: ", all_attrs.shape)
        return all_predictions, all_attrs

def test_attributes(model, loader, config):
    print("Computing attributes...")
    predictions, attr = get_attributes_from_loader(model, loader)
    print("pred shape: ", predictions.shape)
    print("attr shape: ", attr.shape)

    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_score_list = []

    for i in range(0, predictions.shape[1]):
        y_true, y_pred = attr[:, i], predictions[:, i]
        accuracy_list.append(accuracy_score(y_true, y_pred))
        if i == 0: #If it is age
            precision_list.append(precision_score(y_true, y_pred, average='macro'))
            recall_list.append(recall_score(y_true, y_pred, average='macro'))
            f1_score_list.append(f1_score(y_true, y_pred, average='macro'))
        else:
            precision_list.append(precision_score(y_true, y_pred))
            recall_list.append(recall_score(y_true, y_pred))
            f1_score_list.append(f1_score(y_true, y_pred))

    average_acc = np.mean(accuracy_list)
    average_precision = np.mean(precision_list)
    average_recall = np.mean(recall_list)
    average_f1score = np.mean(f1_score_list)

    print("accuracy_list: ", accuracy_list)
    print("precision_list: ", precision_list)
    print("recall_list: ", recall_list)
    print("f1_score_list: ", f1_score_list)

    print("average_acc: ", average_acc)
    print("average_precision: ", average_precision)
    print("average_recall: ", average_recall)
    print("average_f1score: ", average_f1score)

    if config["wandb"]:
            wandb.log({"mAP": mAP})
            wandb.log({"accuracy_list": accuracy_list})
            wandb.log({"precision_list": precision_list})
            wandb.log({"recall_list": recall_list})
            wandb.log({"f1_score_list": f1_score_list})

            wandb.log({"average accuracy": average_acc})
            wandb.log({"average precision": average_precision})
            wandb.log({"average recall": average_recall})
            wandb.log({"average f1": average_f1score})

In [46]:
def test(model, gallery_loader, queries_loader, ground_truth_dict, config, save_model=False):
    print("Testing")
    model.eval()

    test_mAP(model, gallery_loader, queries_loader, ground_truth_dict, config)
    test_attributes(model, gallery_loader, config)

    if save_model :
      # Save the model in the exchangeable ONNX format
      inputs, id, attr = next(iter(gallery_loader))
      torch.onnx.export(model, inputs.to(device), "model.onnx", True)
      if config["wandb"]:
        wandb.save("model.onnx")

In [48]:
model = model_pipeline(config)

{'wandb': False, 'device': 'auto', 'train_root': '/content/dataset/train', 'test_root': '/content/dataset/test', 'queries_root': '/content/dataset/queries', 'attributes_file': '/content/dataset/annotations_train.csv', 'dataset': 'Market1501', 'backbone': 'resnet18', 'split': {'full_training_size': 0.75, 'train_size': 0.8}, 'compose': {'resize_h': 224, 'resize_w': 224}, 'epochs': 1, 'training_batch_size': 8, 'validation_batch_size': 32, 'learning_rate': 0.01, 'weight_decay': 1e-06, 'momentum': 0.9, 'test_before_training': True, 'test_after_epochs': 10, 'mAP_rank': 10}
Extract queries proportion: 0.11
Identities in train set: 563
Identities in validation set: 188
Train set size: 9709
Validation set size: 2919
Number of validation queries: 361


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Number of attributes: 27
Backbone feature size: 512
Using resnet18 as backbone
Testing
Computing gallery features...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Computing query features...


...

Epoch 0: Loss after 00392 examples: 13.255
Epoch 0: Loss after 00792 examples: 11.468
Epoch 0: Loss after 01192 examples: 10.751
Epoch 0: Loss after 01592 examples: 9.443
Epoch 0: Loss after 01992 examples: 9.499
Epoch 0: Loss after 02392 examples: 14.040
Epoch 0: Loss after 02792 examples: 11.197
Epoch 0: Loss after 03192 examples: 9.734
Epoch 0: Loss after 03592 examples: 9.848
Epoch 0: Loss after 03992 examples: 9.361
Epoch 0: Loss after 04392 examples: 9.424
Epoch 0: Loss after 04792 examples: 8.773
Epoch 0: Loss after 05192 examples: 8.822
Epoch 0: Loss after 05592 examples: 8.900
Epoch 0: Loss after 05992 examples: 8.320
Epoch 0: Loss after 06392 examples: 9.404
Epoch 0: Loss after 06792 examples: 9.063
Epoch 0: Loss after 07192 examples: 11.485
Epoch 0: Loss after 07592 examples: 8.296
Epoch 0: Loss after 07992 examples: 7.344
Epoch 0: Loss after 08392 examples: 7.774
Epoch 0: Loss after 08792 examples: 7.368
Epoch 0: Loss after 09192 examples: 9.105
Epoch 0: Loss after 09592 ex


Computing query features...



Computing cosine similarities...
Similarity matrix shape: torch.Size([361, 2919])
current_ground_truth_query: 0
current_ground_truth_query_set: {1377, 66, 1348, 328, 73, 2094, 2319, 2771, 724, 2809, 502, 2585, 955, 1471}
current_predictions_list: [66, 724, 1937, 1377, 2101, 2561, 1746, 1860, 15, 591]
0.19642857142857142
current_ground_truth_query: 1
current_ground_truth_query_set: {2181, 921, 1307, 541, 1185, 1186, 1662, 2861, 1841, 438, 1210, 1597, 829, 446, 581, 1479, 1616, 2772, 2134, 1751, 1117, 2013, 997, 1128, 748, 1645, 375, 632, 1022}
current_predictions_list: [921, 1616, 252, 2481, 997, 1597, 581, 1186, 390, 1022]
0.18727422003284072
current_ground_truth_query: 2
current_ground_truth_query_set: {612, 166, 1735, 238, 1838, 1618, 2647, 1049, 1372}
current_predictions_list: [1949, 1258, 437, 2762, 2814, 2274, 943, 2318, 2703, 1125]
0.0
current_ground_truth_query: 3
current_ground_truth_query_set: {1024, 1255, 40, 781, 334, 946, 146, 2834, 2899, 1563, 1790}
current_predictions_li


pred shape:  (2919, 27)
attr shape:  (2919, 27)
accuracy_list:  [0.8098663926002055, 0.8043850633778691, 0.7591640972935937, 0.8831791709489552, 0.86810551558753, 0.7303871188763275, 0.9383350462487153, 0.7608770126755738, 0.9719081877355259, 0.7372387804042481, 0.894141829393628, 0.8400137033230558, 0.9390202124015073, 0.9575196985268928, 0.9777321000342583, 0.9013360739979445, 0.9201781431997259, 0.9510106200753683, 0.7838300787941076, 0.9280575539568345, 0.9616306954436451, 1.0, 0.9893799246317232, 0.8643371017471737, 0.8674203494347379, 0.9650565262076053, 0.8862624186365193]
precision_list:  [0.20246659815005139, 0.325, 0.13829787234042554, 0.17582417582417584, 0.9064695009242144, 0.9125475285171103, 0.9536100453435647, 0.6113402061855671, 0.0, 0.6527987897125568, 0.6191950464396285, 0.7898966704936854, 0.584717607973422, 0.0, 0.944954128440367, 0.6346153846153846, 1.0, 0.44171779141104295, 0.8052159739201304, 0.36363636363636365, 0.0, 0.0, 0.0, 0.3220338983050847, 0.302904564315

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


In [ ]:
train_set, val_set, val_queries = split_data(config)
model, train_loader, val_loader, val_queries_loader, criterion, optimizer = setup(train_set, val_set, val_queries, config)
id_ground_truth_dict = build_ground_truth(val_set, val_queries)